# BETA_random_eqrthquake_locations_w_pyvista_vtk

Testing the random picking of subsurface eqrthquake locations (moment-tensor locations) and 3D plotting of the picks and model 

### Step 0

Load packages

In [ ]:
#load all packages
import datetime
import pickle
import copy
import os

from pathlib import Path

import numpy as np
import pandas as pd
import pyvista as pv
import matplotlib.pyplot as plt 

from sys import argv

from matplotlib.colors import Normalize
from pyaspect.model.gridmod3d import gridmod3d as gm
from pyaspect.model.bbox import bbox as bb
from pyaspect.model.gm3d_utils import compress_gm3d_to_file
from pyaspect.model.gm3d_utils import decompress_gm3d_from_file
from pyaspect.moment_tensor import MomentTensor
from pyaspect.specfemio.headers import StationHeader
from pyaspect.specfemio.headers import SolutionHeader
from pyaspect.specfemio.headers import CMTSolutionHeader
from pyaspect.specfemio.headers import ForceSolutionHeader
from pyaspect.specfemio.headers import RecordHeader
from pyaspect.specfemio.write import write_cmtsolution
from pyaspect.specfemio.write import write_forcesolution
from pyaspect.specfemio.write import write_grouped_forcesolutions
from pyaspect.specfemio.write import write_stations
from pyaspect.specfemio.write import write_record
from pyaspect.specfemio.write import write_records
from pyaspect.specfemio.read import read_stations
from pyaspect.specfemio.read import read_solution
from pyaspect.specfemio.read import read_cmtsolution
from pyaspect.specfemio.read import read_forcesolution
from pyaspect.specfemio.utils import *

### Step 1 

Extract the ndarray of the subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
data_in_dir  = 'data/output/'
data_out_dir = data_in_dir
!ls {data_in_dir} 

### Step 6 

Decompress the ndarray of the sliced, subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
# set filename then used it to decompress model
ifqn = f'{data_out_dir}/vsliced_subsmp_smth_nam_2017_vp_vs_rho_Q_model_dx100_dy100_dz100_maxdepth5850_sig250.npz'
vslice_gm3d, other_pars = decompress_gm3d_from_file(ifqn)

print()
print('decompressed gridded model\n:',vslice_gm3d) 
print()
print('other parameters:\n',other_pars)
print()

# WARNING: this will unpack all other_pars, if you overwrite a variable of the samename as val(key), then you 
#          may not notice, and this may cause large headaches.  I use it because I am aware of it.
'''
for key in other_pars:
    locals()[key] = other_pars[key]  #this is more advanced python than I think is reasonable for most 
sig_meters = sig
''';

# another way to get these varibles is just use the accessor functions for the gridmod3d.  We need them later.
xmin = other_pars['xmin']
dx   = other_pars['dx']
nx   = other_pars['nx']
ymin = other_pars['ymin']
dy   = other_pars['dy']
ny   = other_pars['ny']
zmin = other_pars['zmin']
dz   = other_pars['dz']
nz   = other_pars['nz']
sig_meters = other_pars['sig']  # this variable is used later
print('sig_meters:',sig_meters)

In [ ]:
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape + 1 because we want to inject our values on
#   the CELL data
nam_dims = list(vslice_gm3d.get_npoints())
nam_origin = [0,0,-vslice_gm3d.get_gorigin()[2]]
#nam_origin = list(vslice_gm3d.get_gorigin())
#nam_origin[2] *= -1
nam_origin = tuple(nam_origin)
nam_spacing = list(vslice_gm3d.get_deltas())
nam_spacing[2] *=-1
nam_spacing = tuple(nam_spacing)
print('nam_dims:',nam_dims)
print('nam_origin:',nam_origin)
print('nam_spacing:',nam_spacing)

# Edit the spatial reference
grid.dimensions = np.array(nam_dims) + 1
grid.origin = nam_origin  # The bottom left corner of the data set
grid.spacing = nam_spacing  # These are the cell sizes along each axis
nam_pvalues = vslice_gm3d.getNPArray()[0]
print('pvalues.shape:',nam_pvalues.shape)

# Add the data values to the cell data
grid.cell_arrays["values"] = nam_pvalues.flatten(order="F")  # Flatten the array!

# Now plot the grid!
cmap = plt.cm.jet
#grid.plot(show_edges=True,cmap=cmap)
grid.plot(cmap=cmap,opacity=1.0)


In [ ]:
slices = grid.slice_orthogonal()

#slices.plot(show_edges=True,cmap=cmap)
slices.plot(cmap=cmap)

## create random virtual source (to specfem stations, but using reciprocity -- sources)

In [ ]:
#coords = vslice_gm3d.getGlobalCoordsPointsXYZ()
coords = vslice_gm3d.getLocalCoordsPointsXYZ()
coords[:,2] = -coords[:,2]

xc = np.unique(coords.T[0,:])
yc = np.unique(coords.T[1,:])
zc = np.unique(coords.T[2,:])


n_rand_p = 1000

#stay away from the edges of the model for derivatives 
# and to avoid boundary effects
xy_pad = 1000 

lrx = np.min(xc) + xy_pad
lry = np.min(yc) + xy_pad
lrz = -4300.0

hrx = np.max(xc) - xy_pad
hry = np.max(yc) - xy_pad
hrz = -3000.0

srx = hrx - lrx
sry = hry - lry
srz = hrz - lrz

r_xyz_list = []
for i in range(n_rand_p):
    rx = lrx + srx*np.random.rand()
    ry = lry + sry*np.random.rand()
    rz = lrz + srz*np.random.rand()
    r_xyz_list.append([rx,ry,rz])
    
r_xyz = np.array(r_xyz_list)
    

#r_xyz = np.vstack(np.meshgrid(rx,ry,rz)).reshape(3,-1).T
print('r_xyz:\n',r_xyz)


In [ ]:
pv_rpoints = pv.wrap(r_xyz)
p = pv.Plotter()
#p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(grid,cmap=cmap,opacity=0.75)
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

In [ ]:
!pwd

In [ ]:
#!mkdir {data_out_dir}/tmp
!ls {data_out_dir}

## make cross (half or full) group for calculating spacial derivatives

In [ ]:
if 'l_grp_stations' in locals() or 'l_grp_stations' in globals():
    print('deleting')
    del l_grp_stations
    
    
# this is the path to the project dir on the cluster
my_proj_dir = '/scratch/seismology/tcullison/test_mesh/Batch_Src_Test'

    
# make a list of station headers for each random location
l_stations = []
for i in range(len(r_xyz)):
    
    #name = 't' + str(i).zfill(len(str(len(r_xyz))))
    tr_bname = 'tr'
    new_s = StationHeader(name=tr_bname,
                          network='NL',
                          lat_yc=r_xyz[i,1],
                          lon_xc=r_xyz[i,0],
                          elevation=0.0,
                          depth=-r_xyz[i,2],
                          trid=i)
    l_stations.append(new_s)
#print('len(l_stats):',len(l_stations))
                                           
# make the group membors for each station above
# function below returns a list[list[]] like structure
m_delta = 250.0 # distance between cross stations for derivatives
assert m_delta < xy_pad #see cells above this is padding
l_grp_stations = make_grouped_half_cross_station_headers(stations=l_stations,delta=m_delta)
name_list = []
for s in flatten_grouped_headers(l_grp_stations):
    name_list.append(f's{str(s.sid).zfill(2)}g{str(s.gid).zfill(2)}t{str(s.trid).zfill(6)}')
    
print('len(name_list):',len(name_list))
print('len(name_set): ',len(set(name_list)))
    

# this is a list[] structure with unique stations (small chance a group member has same coordinate)
s_grp_stations = sorted(copy.deepcopy(flatten_grouped_headers_unique(l_grp_stations)))

# if lengths are the same then all group members are unique
print(f'len(l_grp): {len(flatten_grouped_headers(l_grp_stations))}')
print(f'len(s_grp): {len(s_grp_stations)}')

#flatten the l_grp_stations (NOT guarantied unique!)
l_grp_stations = sorted(flatten_grouped_headers(l_grp_stations))

In [ ]:
write_stations(data_out_dir + '/tmp',l_grp_stations,auto_name=True,auto_network=True)

# For records only!
#write_stations(data_out_dir + '/tmp',l_grp_stations,write_h=False) 

In [ ]:
!ls -ltrh {data_out_dir + '/tmp'} 

In [ ]:
!head {data_out_dir}/tmp/STATIONS

In [ ]:
!tail {data_out_dir}/tmp/STATIONS

In [ ]:
fqp = data_out_dir + '/tmp'
rw_stations = sorted(read_stations(fqp))

print('stations equal?:', rw_stations == l_grp_stations)

In [ ]:
all_g_xyz = get_xyz_coords_from_station_list(rw_stations)
all_g_xyz[:,2] *= -1 #pyview z-up positive and oposize sign of standard geophysics 
pv_all_points = pv.wrap(all_g_xyz)
p = pv.Plotter()
p.add_mesh(grid,cmap=cmap,opacity=0.75)
p.add_mesh(pv_all_points, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

## make random virtual recievers locations (solutions/sources in specfem)

In [ ]:
#coords = vslice_gm3d.getGlobalCoordsPointsXYZ()
coords = vslice_gm3d.getLocalCoordsPointsXYZ()
coords[:,2] = -coords[:,2]

xc = np.unique(coords.T[0,:])
yc = np.unique(coords.T[1,:])
zc = np.unique(coords.T[2,:])


n_rand_p = 5
rz = -500

lrx = np.min(xc)
lry = np.min(yc)

hrx = np.max(xc)
hry = np.max(yc)

srx = hrx - lrx
sry = hry - lry

s_xyz_list = []
for i in range(n_rand_p):
    rx = lrx + srx*np.random.rand()
    ry = lry + sry*np.random.rand()
    s_xyz_list.append([rx,ry,rz])
    
s_xyz = np.array(s_xyz_list)
    

print('s_xyz:\n',s_xyz)


In [ ]:
pv_spoints = pv.wrap(s_xyz)
p = pv.Plotter()
#p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(grid,cmap=cmap,opacity=0.3)
p.add_mesh(pv_spoints, render_points_as_spheres=True, point_size=8,opacity=1,color='red')
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

## make force solution headers

In [ ]:
# make a list of station headers for each random location
l_solutions = []
for i in range(len(s_xyz)):
    
    #NOTE!!!! the depth is set to the NEGATIVE (makes it positive) due to sign convention
    new_s = ForceSolutionHeader(ename=f'Event-{str(i).zfill(4)}',
                                lat_yc=s_xyz[i,1],
                                lon_xc=s_xyz[i,0],
                                depth=-s_xyz[i,2],
                                tshift=0.0,
                                date=datetime.datetime.now(),
                                f0=0.0,
                                factor_fs=1,
                                comp_src_EX=1,
                                comp_src_NY=0,
                                comp_src_Zup=0,
                                proj_id=0,
                                eid=i,
                                sid=0)
    l_solutions.append(new_s)

## make solution group

In [ ]:
l_grp_solutions_h = make_grouped_triplet_force_solution_headers(solutions=l_solutions)
grp_s_xyz = get_xyz_coords_from_solution_list(flatten_grouped_headers(l_grp_solutions_h))

for s in flatten_grouped_headers(l_grp_solutions_h):
    print(f'solution:\n{s}')
    print()
print(f'coords:\n{grp_s_xyz}')
print()

In [ ]:
for grp_s in l_grp_solutions_h:
    p = os.path.join(data_out_dir, 'tmp')
    print(p)
    p = os.path.join(p, f'run{str(grp_s[0].eid+1).zfill(4)}')
    print(p)
    p = os.path.join(p, 'DATA')
    print(p)
    #p = f'{data_out_dir}/tmp/run{str(grp_s[0].eid+1).zfill(4)}'
    #print(p)
    Path(p).mkdir(parents=True, exist_ok=True)

In [ ]:
!ls {data_out_dir}/tmp
!pwd

In [ ]:
for grp_s in l_grp_solutions_h:
    write_grouped_forcesolutions(f'{data_out_dir}/tmp/run{str(grp_s[0].eid+1).zfill(4)}',grp_s)
    #write_grouped_forcesolutions(f'{data_out_dir}/tmp/run{str(grp_s[0].eid+1).zfill(4)}',grp_s,write_h=False)
    #print(f'{data_out_dir}/tmp/run{str(grp_s[0].eid+1).zfill(4)}')
    #for s in grp_s:
        #print(f's:\n{s}')


In [ ]:
!ls -l {data_out_dir}tmp/run*/DATA

## make record from solutions and stations

In [ ]:
# make stations list per solution_group 
poor_l_records = []

for i in range(len(l_grp_solutions_h)):
    grp_s = l_grp_solutions_h[i]
    l_force_stations = []
    for sol in grp_s:
        l_stat = copy.deepcopy(l_grp_stations)
        for s in l_stat:
            s.eid = sol.eid
            s.sid = sol.sid
        l_force_stations += l_stat

    record = RecordHeader(solutions_h=grp_s,stations_h=l_force_stations,rid=i)
    poor_l_records.append(record)

src_grouped_stations = []
for sgrp in l_grp_solutions_h:
    l_rgrp = []
    for s in sgrp:
        l_rgrp.append(copy.deepcopy(l_grp_stations))
    src_grouped_stations.append(l_rgrp)

print(f'len(l_grp_solutions_h):{len(l_grp_solutions_h)}')
print(f'len(l_grp_solutions_h[0]):{len(l_grp_solutions_h[0])}')
print(f'len(l_grp_solutions_h[0][0]):{len(l_grp_solutions_h[0][0])}')
print(f'len(src_grouped_stations):{len(src_grouped_stations)}')
print(f'len(src_grouped_stations[0]):{len(src_grouped_stations[0])}')
l_records = []
l_records = make_records(l_src=l_grp_solutions_h,l_rec=src_grouped_stations)
    
print(f'len(l_records): {len(l_records)}')
print()

for i in range(len(l_records)):
    print(f'rec[{i}] == prec[{i}]: {l_records[i] == poor_l_records[i]}')

print('Print Out of ALL Records\n')
for i in range(len(l_records)):
    r = l_records[i]
    p = poor_l_records[i]
    print(f'Record-{r.rid}:\n{r}')
    print()
    print('---------------------------------------------------------------------------')
    print()

## pickle record list

In [ ]:
f = open(f'{data_out_dir}/tmp/project_record_list','wb')
pickle.dump(l_records,f)
f.close()

!ls -ltrh {data_out_dir}/tmp/project_record_list

In [ ]:
f = open(f'{data_out_dir}/tmp/project_record_list','rb')
dill_l_records = pickle.load(f)
f.close

print('Check all records:')
print()
for i in range(len(dill_l_records)):
    print(f'Dill_Rec == Orig_Rec?: {dill_l_records == l_records}')

## write records and headers in records:

In [ ]:

proj_fqp = f'{data_out_dir}tmp/' 
#write_record(proj_fqp,dill_l_records[0],fname='test_project.record')
write_records(proj_fqp,dill_l_records,fname='test_project_record',auto_name=True,auto_network=True)

In [ ]:
!ls -ltrh data/output/tmp/
print()
!ls -ltrh {data_out_dir}tmp/run*/Data

In [ ]:
for r in dill_l_records:
    print(f'Record:\n{r}')

In [ ]:
from pyaspect.specfemio.utils import _join_path_fname
test_proj_name = 'FirstTestProject'
test_proj_fqp =  os.path.join(data_out_dir, 'tmp/TestProjects')
test_spec_fqp = '/quanta1/home/tcullison/DevGPU_specfem3d'
test_pyutils_fqp = '/quanta1/home/tcullison/myscripts/specfem/pyutils'
test_script_fqp = '/quanta1/home/tcullison/myscripts/specfem'
print(f'test_proj_fqp: {test_proj_fqp}')
!ls {test_proj_fqp}

In [ ]:
from pyaspect.project import make_project
#make_records(l_src=l_grp_solutions_h,l_rec=src_grouped_stations)
make_project(test_proj_name,
             test_proj_fqp,
             test_spec_fqp,
             test_pyutils_fqp,
             test_script_fqp,
             l_grp_solutions_h,
             src_grouped_stations)

In [ ]:
ls -ltrh data/output/tmp/TestProjects/FirstTestProject/

In [ ]:
cat ~/Seismic/myscratch/test_mesh/NN_Batch_Src_Test_E5_Stub/run0001/DATA/Par_file

In [ ]:
x = goobleydoobleydoo('42')

In [ ]:
def sillyfunc(index=None):
    if index == None:
        print('none')
    else:
        print(f'index={index}')
        
sillyfunc(2)

In [ ]:
df_rec = []
for i in range(6):
    df_rec.append({'sid':i%3,'eid':0})
df = pd.DataFrame.from_records(df_rec)
print(df)
print()

print(df['sid'])
print()
s = set(df['sid'])
ss = set(df['sid'])
print(s)
print(f's == ss: {s == ss}')